# 1 参数设定

In [1]:
import torch
import numpy as np


class Config():
    def __init__(self):
        self.mode = 'train' # train or test


        # 数据的部分特征
        self.data_lable = 'pig'
        self.data_path = './sketch_datas/'
        self.data_labels_list = ['ambulance','apple','bear','bicycle','bird','bus','cat','foot','owl','pig']
        self.data_max = 1500 # 每个类别的最大数据量,节省训练时间
        # self.full_length = 256 # 一条数据的完整长度
        self.N_max = 0 # 一条数据的最大长度，可修改
        self.lengths = [] # 一条数据的实际长度，可修改


        # 编码器的参数设置
        self.encoder_dim = 256 # encoder的输出维度
        self.z_dim = 128 # 中间向量Z的维度

        # 解码器的参数设置
        self.decoder_dim = 512 # decoder的输出维度
        self.M = 20 # GMM的分布数量

        # 训练参数
        if torch.cuda.is_available():
            self.device = 'cuda'
        elif torch.backends.mps.is_available():
            self.device = 'mps'
        else:
            self.device = 'cpu'

        self.grad_clip = 1.0 # 梯度裁剪
        self.batch_size = 16
        self.lr = 0.001
        self.epoch = 10
        self.W_kl = 0.5 # kl损失的权重
        self.temperature = 0.4 # gumbel softmax的温度参数
        





# 2 数据
读入数据，并对数据进行预处理
数据集由配置文件决定

In [2]:
import random
from re import split

def get_max(arr):
    return arr.shape[1]
    

# 按照7:1:2划分数据集
def split_data(arr, ratios):
    # 计算分割点
    total_length = arr.shape[0]
    splits = [int(total_length * ratio / sum(ratios)) for ratio in ratios]
    
    splits[-1] = total_length - sum(splits[:-1])

    # 按照分割点分割数组
    split_arrays = []
    start = 0
    for split in splits:
        split_arrays.append(arr[start:start + split])
        start += split

    return split_arrays

# 按照batch_size随机划分数据集
def split_in_batch(arr, config):
    # 划分batch，不足batchsize的直接丢弃
    batch_num = arr.shape[0] // config.batch_size 
    max_length = batch_num * config.batch_size # 最长有效数据长度

    # 打乱数据
    shuffled_arr = arr[:max_length].copy()
    np.random.shuffle(shuffled_arr)

    split_arrays = np.array(np.split(shuffled_arr, batch_num))

    return split_arrays


In [3]:
config = Config() # 初始化相关配置


f_path = config.data_path + config.data_lable + '.npy'
f = np.load(f_path)[:config.data_max] # 最多只用1000条数据训练
config.N_max = get_max(f) # 获取最大长度，数据已补全
train_data, val_data, test_data = split_data(f,[8,1,1])

train_data = split_in_batch(train_data, config).astype(np.float32)
val_data = split_in_batch(val_data, config).astype(np.float32)
test_data = split_in_batch(test_data, config).astype(np.float32)


# length列表
config.lengths = [config.N_max] * config.batch_size 

print(config.data_lable, 'dataset is loaded')
print('N_max:',config.N_max,',data_type:', train_data.dtype,', batch_size:',config.batch_size)
print('train_data:',train_data.shape)
print('val_data:',val_data.shape)
print('test_data:',test_data.shape)


pig dataset is loaded
N_max: 151 ,data_type: float32 , batch_size: 16
train_data: (75, 16, 151, 5)
val_data: (9, 16, 151, 5)
test_data: (9, 16, 151, 5)


# 3 模型搭建

In [4]:
import torch.nn as nn
import torch

# 2 

class encoderRNN(nn.Module):
    def __init__(self, config):
        super(encoderRNN, self).__init__()
        self.lstm = nn.LSTM(5, config.encoder_dim, bidirectional=True) # 双向lstm
        self.fc_mu = nn.Linear(config.encoder_dim*2, config.z_dim)
        self.fc_sigma = nn.Linear(config.encoder_dim*2, config.z_dim)


    def forward(self, config, input):
        h0 = torch.zeros(2, config.batch_size, config.encoder_dim).to(config.device)
        c0 = torch.zeros(2, config.batch_size, config.encoder_dim).to(config.device)
        hidden_cell = (h0, c0)

        outputs, (hidden, cell) = self.lstm(input, hidden_cell)

        # 完成编码，调整数据形状
        # (2, batch_size, hidden_size) -> (batch_size, 2*hidden_size)
        hidden_forward, hidden_backward = torch.split(hidden, 1, 0)
        hidden_cat = torch.cat([hidden_forward.squeeze(0), hidden_backward.squeeze(0)], 1)
        
        # mu部分
        mu = self.fc_mu(hidden_cat)

        # sigma部分
        sigma_hat = self.fc_sigma(hidden_cat)
        sigma = torch.exp(sigma_hat / 2.0)

        # gaussian noise
        N = torch.randn_like(mu)

        Z = mu + sigma * N


        return Z, mu, sigma_hat

# 2
class decoderRNN(nn.Module):
    def __init__(self, config):
        super(decoderRNN, self).__init__()
        self.N_max = config.N_max
        
        self.lstm = nn.LSTM(config.z_dim + 5, config.decoder_dim)
        self.fc_hc = nn.Linear(config.z_dim, 2 * config.decoder_dim)

        # 6M+3
        self.fc_output = nn.Linear(config.decoder_dim, 6 * config.M + 3)
        # self.fc_q = nn.Linear(config.decoder_dim, 3)
        

    def forward(self, config, input, z):
        h0, c0 = torch.split(torch.tanh(self.fc_hc(z)), config.decoder_dim, 1)
        hidden_cell = ((h0.unsqueeze(0).contiguous()), c0.unsqueeze(0).contiguous()) # [batch_size, lstm_size] -> [1, batch_size, lstm_size]

        output, (hidden, cell) = self.lstm(input, hidden_cell)

        y = self.fc_output(output.view(-1, config.decoder_dim))

        # 分割
        params = torch.split(y, 6, dim=1)
        params_mixture = torch.stack(params[:-1])
        params_pen = params[-1]

        pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy = torch.split(params_mixture, 1, dim=2)

        # pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy = torch.split(self.fc_xy(output), config.M, 2) # 6M
        # q = self.fc_q(output) # 3

        # 后处理 
        # 这些变量的维度？
        pi = nn.functional.softmax(pi.transpose(0, 1).squeeze(), dim=-1).view(self.N_max+1, -1, config.M)
        sigma_x = torch.exp(sigma_x.transpose(0, 1).squeeze()).view(self.N_max+1, -1, config.M)
        sigma_y = torch.exp(sigma_y.transpose(0, 1).squeeze()).view(self.N_max+1, -1, config.M)
        rho_xy = torch.tanh(rho_xy.transpose(0, 1).squeeze()).view(self.N_max+1, -1, config.M)
        mu_x = mu_x.transpose(0, 1).squeeze().contiguous().view(self.N_max+1, -1, config.M)
        mu_y = mu_y.transpose(0, 1).squeeze().contiguous().view(self.N_max+1, -1, config.M)
        q = nn.functional.softmax(params_pen, dim=-1).view(self.N_max+1, -1, 3)

        return pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy, q, hidden, cell



# 4 训练设定

In [5]:
from tqdm import tqdm
from torch import optim
from torch.nn.utils.clip_grad import clip_grad_norm_
import matplotlib.pyplot as plt

# 1
class model(nn.Module):
    def __init__(self, config):
        super(model, self).__init__()
        self.encoder = encoderRNN(config).to(config.device)
        self.decoder = decoderRNN(config).to(config.device)

        self.encoder_optimizer = optim.Adam(self.encoder.parameters(), lr=config.lr)
        self.decoder_optimizer = optim.Adam(self.decoder.parameters(), lr=config.lr)

    
    # KL Loss
    def KL_loss(self, mu, sigma, config):
        return -0.5 * torch.sum(1 + sigma - mu.pow(2) - torch.exp(sigma)) / float(config.z_dim * config.batch_size)

    # Reconstruction Loss
    def RC_loss(self, pi, mu_x, mu_y, sigma_x, sigma_y, rho_xy, q, mask, dx, dy, p, config):
        # pdf
        z_x = ((dx - mu_x) / sigma_x) ** 2
        z_y = ((dy - mu_y) / sigma_y) ** 2
        z_xy = (dx - mu_x) * (dy - mu_y) / (sigma_x * sigma_y)
        z = z_x + z_y - 2 * rho_xy * z_xy
        exp = torch.exp(-z / (2 * (1 - rho_xy ** 2)))
        norm = 2 * np.pi * sigma_x * sigma_y * torch.sqrt(1 - rho_xy ** 2)
        pdf = torch.nan_to_num(exp / norm, 0.0)

        loss_LS = -torch.sum(mask * torch.log(1e-5 + torch.sum(pi * pdf, dim=2))) / float(config.N_max * config.batch_size) # XY
        loss_LP = -torch.sum(p * torch.log(q)) / float(config.N_max * config.batch_size) # PPP

        return loss_LS + loss_LP

    # 处理数据标签（补全，记录有效数据的长度mask机制，分割“笔画、笔状态数据”）
    def make_target(self, config, batch):
        eos = torch.stack([torch.Tensor([0, 0, 0, 0, 1])] * batch.size()[1]).to(config.device).unsqueeze(0)
        batch = torch.cat([batch, eos], 0)
        # 记录有效数据的长度mask机制
        mask = torch.zeros(config.N_max + 1, batch.shape[1]).to(config.device) 
        for idx, length in enumerate(config.lengths):
            mask[:length, idx] = 1 # 真实有效数据才能进行后续计算

        # 分割“笔画、笔状态数据”
        dx = torch.stack([batch.data[:, :, 0]] * config.M, 2)
        dy = torch.stack([batch.data[:, :, 1]] * config.M, 2)
        p1 = batch.data[:, :, 2]
        p2 = batch.data[:, :, 3]
        p3 = batch.data[:, :, 4]
        p = torch.stack([p1, p2, p3], dim=2)

        return mask, dx, dy, p


        

    # 储存模型（解码编码分开保存）
    def save(self, config, model):
        # save encoder and decoder
        torch.save(model.encoder.state_dict(), f"trained_SRNN_{config.data_lable}_encoder.pth")
        torch.save(model.decoder.state_dict(), f"trained_SRNN_{config.data_lable}_decoder.pth")


    def train(self, config, train_data, val_data):
        train_loss_epoch_history = []
        train_LKL_epoch_history = []
        train_LR_epoch_history = []

        val_loss_epoch_history = []
        val_LKL_epoch_history = []
        val_LR_epoch_history = []


        best_val_loss = 100000000 # 初始化一个较大的值
        best_model = None
        
        train_data = torch.from_numpy(train_data).to(config.device)
        val_data = torch.from_numpy(val_data).to(config.device)


        for epoch in range(config.epoch):
            print(f'Start training, now epoch {epoch}',end = '\t')
            train_loss_history = []
            train_LKL_history = []
            train_LR_history = []

            val_loss_history = []
            val_LKL_history = []
            val_LR_history = []



            # 训练阶段
            self.encoder.train()
            self.decoder.train()

            for train_batch in tqdm(train_data): 
                # encoder
                train_batch = train_batch.permute(1,0,2)
                train_z, train_mu, train_sigma = self.encoder(config, train_batch)

                # decoder
                # S0数据准备
                s0 = torch.stack([torch.Tensor([0, 0, 1, 0, 0])] * config.batch_size).to(config.device).unsqueeze(0) # [1, batch_size, 5]

                train_batch_init = torch.cat([s0, train_batch], dim = 0) # 引入s0的数据,[N_max+1, batch_size, 5]
                train_z_stack = torch.stack([train_z] * (config.N_max + 1)) # [N_max+1, batch_size, z_dim]
                train_inputs = torch.cat([train_batch_init, train_z_stack], dim=2) # [N_max+1, batch_size, 5+z_dim]

                train_pi, train_mu_x, train_mu_y, train_sigma_x, train_sigma_y, train_rho_xy, train_q, _, _ = self.decoder(config, train_inputs, train_z)

                # 计算损失
                train_mask, train_dx, train_dy, train_p = self.make_target(config, train_batch) # y_true数据
             
                rc_loss = self.RC_loss(train_pi, train_mu_x, train_mu_y, train_sigma_x, train_sigma_y, train_rho_xy, train_q, train_mask, train_dx, train_dy, train_p, config)
                kl_loss = self.KL_loss(train_mu, train_sigma, config)
                train_loss = rc_loss + config.W_kl *kl_loss 

                # 记录训练损失
                train_loss_history.append(train_loss.item())
                train_LKL_history.append(kl_loss.item())
                train_LR_history.append(rc_loss.item())


                # 反向传播
                self.encoder_optimizer.zero_grad()
                self.decoder_optimizer.zero_grad()
                train_loss.backward()
                
                clip_grad_norm_(self.encoder.parameters(), config.grad_clip)
                clip_grad_norm_(self.decoder.parameters(), config.grad_clip)
                self.encoder_optimizer.step()
                self.decoder_optimizer.step()


                # 验证val阶段
                self.encoder.eval()
                self.decoder.eval()

                with torch.no_grad():
                    for val_batch in val_data:
                        # encoder
                        val_batch = val_batch.permute(1,0,2)
                        val_z, val_mu, val_sigma = self.encoder(config, val_batch)

                        # decoder

                        val_s0 = torch.stack([torch.Tensor([0, 0, 1, 0, 0])] * config.batch_size).to(config.device).unsqueeze(0) # [1, batch_size, 5]

                        val_batch_init = torch.cat([val_s0, val_batch], dim = 0)
                        val_z_stack = torch.stack([val_z] * (config.N_max + 1))
                        val_inputs = torch.cat([val_batch_init, val_z_stack], dim=2)

                        val_pi, val_mu_x, val_mu_y, val_sigma_x, val_sigma_y, val_rho_xy, val_q, _, _ = self.decoder(config, val_inputs, val_z)

                        # 计算损失
                        val_mask, val_dx, val_dy, val_p = self.make_target(config, val_batch) # y_true数据

                        val_rc_loss = self.RC_loss(val_pi, val_mu_x, val_mu_y, val_sigma_x, val_sigma_y, val_rho_xy, val_q, val_mask, val_dx, val_dy, val_p, config)
                        val_kl_loss = self.KL_loss(val_mu, val_sigma, config)
                        val_loss = val_rc_loss + config.W_kl * val_kl_loss

                        # 记录验证损失
                        val_loss_history.append(val_loss.item())
                        val_LKL_history.append(val_kl_loss.item())
                        val_LR_history.append(val_rc_loss.item())


            # 记录每个epoch的损失
            train_loss_epoch_history.append(np.mean(train_loss_history).item())
            train_LKL_epoch_history.append(np.mean(train_LKL_history).item())
            train_LR_epoch_history.append(np.mean(train_LR_history).item())

            val_loss_epoch_history.append(np.mean(val_loss_history).item())
            val_LKL_epoch_history.append(np.mean(val_LKL_history).item())
            val_LR_epoch_history.append(np.mean(val_LR_history).item())


            # 保存最优的模型
            if best_val_loss > min(val_loss_epoch_history):
                best_model = self # 更新
                
        # end of epoch for

        # 保存模型
        self.save(config, best_model)

        # 展示训练过程
        plt.figure()
        plt.subplot(1, 2, 1)
        plt.plot(train_loss_epoch_history, label='train loss')
        plt.plot(train_LR_epoch_history, label='train reconstruction loss')
        plt.plot(train_LKL_epoch_history, label='train KL_loss')
        plt.title(f'Train Loss')
        plt.legend()
        plt.grid(True)

        # save val loss graph
        plt.figure()
        plt.subplot(1, 2, 2)
        plt.plot(val_loss_epoch_history, label='val loss')
        plt.plot(val_LR_epoch_history, label='val reconstruction loss')
        plt.plot(val_LKL_epoch_history, label='val KL_loss')
        plt.title(f'Validation Loss')
        plt.legend()
        plt.grid(True)

        plt.show()


    # 使用测试集数据测试模型（展示生成结果）
    def test(self, config, test_data):
        pass
        


        # 最终生成样例(有条件)
        # self.generate(config, test_data)





# 5 训练

In [6]:
my_model = model(config)

my_model.train(config, train_data, val_data)

# my_model.test(config, test_data)


Start training, now epoch 0	

100%|██████████| 75/75 [00:30<00:00,  2.44it/s]


Start training, now epoch 1	

100%|██████████| 75/75 [00:29<00:00,  2.51it/s]


Start training, now epoch 2	

 57%|█████▋    | 43/75 [00:16<00:12,  2.59it/s]

# 6 展示